In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from scipy.stats import levy
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler




In [2]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
df = pd.read_csv('Telco-Customer-Churn.csv')
# Check for missing values
print(df.isnull().sum())
# Remove duplicates
df.drop_duplicates(inplace=True)


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
print(df.columns)
# Assuming 'df' is your DataFrame and you want to drop the column named 'customerID'
df.drop(columns=['customerID'], inplace=True)
# If leading or trailing whitespaces are present, remove them
df.columns = df.columns.str.strip()


Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


In [6]:
# Assuming df is your DataFrame and "TotalCharges" column contains float numbers

# Convert "TotalCharges" column to numeric, coerce errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill NaN values with 0 (or any other suitable value)
df['TotalCharges'].fillna(0, inplace=True)

# Convert float numbers to integers
df['TotalCharges'] = df['TotalCharges'].astype(int)
# Assuming df is your DataFrame and "MonthlyCharges" column contains float numbers

# Convert "MonthlyCharges" column to numeric, coerce errors to NaN
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')

# Fill NaN values with 0 (or any other suitable value)
df['MonthlyCharges'].fillna(0, inplace=True)

# Convert float numbers to integers
df['MonthlyCharges'] = df['MonthlyCharges'].astype(int)



In [7]:
encoder = LabelEncoder()
# Encoding target variable 'Churn'
y = encoder.fit_transform(df["Churn"])
df.drop(['Churn'], axis=1, inplace=True)  # Drop the "Churn" column from the DataFrame
X = df.copy()
# Encoding categorical variables

X['gender'] = encoder.fit_transform(X['gender'])
X['Partner'] = encoder.fit_transform(X['Partner'])
X['Dependents'] = encoder.fit_transform(X['Dependents'])
X['PhoneService'] = encoder.fit_transform(X['PhoneService'])
X['MultipleLines'] = encoder.fit_transform(X['MultipleLines'])
X['InternetService'] = encoder.fit_transform(X['InternetService'])
X['OnlineSecurity'] = encoder.fit_transform(X['OnlineSecurity'])
X['OnlineBackup'] = encoder.fit_transform(X['OnlineBackup'])
X['DeviceProtection'] = encoder.fit_transform(X['DeviceProtection'])
X['TechSupport'] = encoder.fit_transform(X['TechSupport'])
X['StreamingTV'] = encoder.fit_transform(X['StreamingTV'])
X['StreamingMovies'] = encoder.fit_transform(X['StreamingMovies'])
X['Contract'] = encoder.fit_transform(X['Contract'])
X['PaperlessBilling'] = encoder.fit_transform(X['PaperlessBilling'])
X['PaymentMethod'] = encoder.fit_transform(X['PaymentMethod'])
# Print features (X)
print("Features (X):")
print(X.head())

# Print target variable (y)
print("\nTarget Variable (y):")
print(y)



Features (X):
   gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0       0              0        1           0       1             0   
1       1              0        0           0      34             1   
2       1              0        0           0       2             1   
3       1              0        0           0      45             0   
4       0              0        0           0       2             1   

   MultipleLines  InternetService  OnlineSecurity  OnlineBackup  \
0              1                0               0             2   
1              0                0               2             0   
2              0                0               2             2   
3              1                0               2             0   
4              0                1               0             0   

   DeviceProtection  TechSupport  StreamingTV  StreamingMovies  Contract  \
0                 0            0            0                0         0   
1   

In [8]:
#SVM ACCURACY BEFORE TRAINING AND TESTING

In [9]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the SVM model
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

# Testing the model
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy * 100, "%")

Accuracy: 0.7352732434350603
              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1036
           1       0.00      0.00      0.00       373

    accuracy                           0.74      1409
   macro avg       0.37      0.50      0.42      1409
weighted avg       0.54      0.74      0.62      1409

Accuracy: 73.52732434350602 %


In [10]:
#SVM ACCURACY AFTER FEATURE EXTRACTION USING COCCO SEARCH ALGORITHM AND THEN SVM TRAINING AND TESTING

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from scipy.spatial.distance import euclidean

# Cuckoo Search Algorithm
def initialize_nests(dim, n):
    return np.random.rand(n, dim)

def get_objective(solution):
    # Placeholder for your objective function
    # This function should return a scalar value indicating the fitness of the solution
    return np.sum(solution)

def get_best_nest(nests, obj_values):
    return nests[np.argmin(obj_values)]

def levy_flight(beta=1.5):
    sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) / (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = np.random.randn() * sigma
    v = np.random.randn()
    step = u / abs(v) ** (1 / beta)
    return step

def cuckoo_search(n, dim, max_iter):
    nests = initialize_nests(dim, n)
    obj_values = [get_objective(nest) for nest in nests]
    best_nest = get_best_nest(nests, obj_values)
    best_obj_value = get_objective(best_nest)

    for _ in range(max_iter):
        new_nests = nests.copy()
        for i, nest in enumerate(new_nests):
            step_size = levy_flight()
            random_nest_index = np.random.randint(n)
            random_nest = nests[random_nest_index]
            step = step_size * (nest - random_nest)
            new_nests[i] += step
            new_nests[i] = np.clip(new_nests[i], 0, 1)  # Ensure solutions are within bounds
        obj_values = [get_objective(new_nest) for new_nest in new_nests]
        new_best_nest = get_best_nest(new_nests, obj_values)
        new_best_obj_value = get_objective(new_best_nest)
        if new_best_obj_value < best_obj_value:
            best_nest = new_best_nest
            best_obj_value = new_best_obj_value
        nests = new_nests
    return best_nest

# Example dataset (replace this with your dataset)
data = load_iris()
X, y = data.data, data.target

# Splitting dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature selection using Cuckoo Search
n_features = X_train.shape[1]
n_cuckoos = 10  # Number of cuckoos (population size)
max_iterations = 100  # Maximum number of iterations

# Selecting the best features





best_feature_indices = cuckoo_search(n_cuckoos, n_features, max_iterations)
best_feature_indices = [int(round(idx)) for idx in best_feature_indices]
# Selecting the best features
X_train_selected = X_train[:, best_feature_indices]
X_test_selected = X_test[:, best_feature_indices]

# Training SVM model with the selected features
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_selected, y_train)

# Testing SVM model
y_pred = svm_model.predict(X_test_selected)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy)
print("Accuracy:", accuracy * 100, "%")


              precision    recall  f1-score   support

           0       1.00      0.68      0.81        19
           1       0.50      0.62      0.55        13
           2       0.69      0.85      0.76        13

    accuracy                           0.71        45
   macro avg       0.73      0.72      0.71        45
weighted avg       0.77      0.71      0.72        45

Accuracy: 0.7111111111111111
Accuracy: 71.11111111111111 %


In [12]:
#SVM ACCURACY AFTER FEATURE EXTRACTION USING FIREFLY  ALGORITHM AND THEN SVM TRAINING AND TESTING

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_iris
from scipy.spatial.distance import euclidean

# Firefly Algorithm
def initialize_fireflies(dim, n):
    return np.random.rand(n, dim)

def attractiveness(beta, distance):
    return np.exp(-beta * distance)

def move_firefly(fireflies, beta):
    for i, f1 in enumerate(fireflies):
        for j, f2 in enumerate(fireflies):
            if attractiveness(beta, euclidean(f1, f2)) > attractiveness(beta, euclidean(fireflies[i], fireflies[j])):
                fireflies[i] += (fireflies[j] - fireflies[i]) * np.random.uniform(0, 1)
    return fireflies

# Example dataset (replace this with your dataset)
data = load_iris()
X, y = data.data, data.target

# Splitting dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature selection using Firefly Algorithm
n_features = X_train.shape[1]
n_fireflies = 10  # Number of fireflies (population size)
max_iterations = 100  # Maximum number of iterations
beta = 1  # Attraction coefficient

# Selecting the best features
fireflies = initialize_fireflies(n_features, n_fireflies)
for _ in range(max_iterations):
    fireflies = move_firefly(fireflies, beta)
best_firefly = fireflies[np.argmin([np.sum(firefly) for firefly in fireflies])]
best_feature_indices = np.where(best_firefly > 0.5)[0]

# Ensure at least 2 best features are selected
selected_indices = np.where(best_firefly > 0.5)[0]
if len(selected_indices) < 2:
    
    sorted_indices = np.argsort(best_firefly)[::-1]
    best_feature_indices = np.zeros_like(best_firefly)
    best_feature_indices[sorted_indices[:2]] = 1

# Selecting the best features
X_train_selected = X_train[:, best_feature_indices > 0.5]
X_test_selected = X_test[:, best_feature_indices > 0.5]

# Training SVM model with the selected features
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_selected, y_train)

# Testing SVM model
y_pred = svm_model.predict(X_test_selected)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy * 100, "%")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

Accuracy: 100.0 %


In [14]:
#SVM ACCURACY AFTER FEATURE EXTRACTION USING GREY-WOLF ALGORITHM AND THEN SVM TRAINING AND TESTING

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from scipy.spatial.distance import euclidean

# Grey Wolf Optimization Algorithm
class GreyWolfOptimizer:
    def __init__(self, n_features, n_wolves=5, max_iter=100, a=2, b=1):
        self.n_features = n_features
        self.n_wolves = n_wolves
        self.max_iter = max_iter
        self.a = a
        self.b = b

    def initialize_wolves(self):
        return np.random.rand(self.n_wolves, self.n_features)

    def fitness(self, wolf):
        return np.sum(wolf)

    def update_position(self, wolf, alpha, beta, delta):
        c1 = 2 * np.random.rand(self.n_features)
        c2 = 2 * np.random.rand(self.n_features)
        c3 = 2 * np.random.rand(self.n_features)
        D_alpha = np.abs(c1 * alpha - wolf)
        D_beta = np.abs(c2 * beta - wolf)
        D_delta = np.abs(c3 * delta - wolf)
        new_position = (alpha - self.a * D_alpha) + (beta - self.b * D_beta) + (delta - self.a * D_delta)
        return new_position

    def optimize(self, X_train, y_train):
        wolves = self.initialize_wolves()
        for _ in range(self.max_iter):
            fitness_values = [self.fitness(wolf) for wolf in wolves]
            alpha_index = np.argmin(fitness_values)
            beta_index = np.argsort(fitness_values)[1]
            delta_index = np.argsort(fitness_values)[2]

            alpha_wolf = wolves[alpha_index]
            beta_wolf = wolves[beta_index]
            delta_wolf = wolves[delta_index]

            for i, wolf in enumerate(wolves):
                wolves[i] = self.update_position(wolf, alpha_wolf, beta_wolf, delta_wolf)

        best_wolf_index = np.argmin([self.fitness(wolf) for wolf in wolves])
        best_wolf = wolves[best_wolf_index]
        return best_wolf

# Example dataset (replace this with your dataset)
data = load_iris()
X, y = data.data, data.target

# Splitting dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature selection using Grey Wolf Optimization Algorithm
n_features = X_train.shape[1]
gwo = GreyWolfOptimizer(n_features=n_features)
best_feature_indices = gwo.optimize(X_train, y_train)

# Ensure at least 2 best features are selected
selected_indices = np.where(best_feature_indices > 0.5)[0]
if len(selected_indices) < 2:
    
    best_feature_indices = gwo.optimize(X_train, y_train)
    selected_indices = np.argsort(best_feature_indices)[-2:]  # Select top 2 features if still less than 2 are selected

best_feature_indices = np.zeros(n_features)
best_feature_indices[selected_indices] = 1

# Selecting the best features
X_train_selected = X_train[:, best_feature_indices > 0.5]
X_test_selected = X_test[:, best_feature_indices > 0.5]

# Training SVM model with the selected features
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_selected, y_train)

# Testing SVM model
y_pred = svm_model.predict(X_test_selected)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy * 100, "%")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

Accuracy: 100.0 %
